In [ ]:
import os

import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F
import torch.optim as optim


from datetime import datetime
from torch import nn
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import MessagePassing, DataParallel
from torch_scatter import scatter
from torch.utils.tensorboard import SummaryWriter

from DataClasses import lmdb_dataset, Dataset, DataListLoader
from ModelFunctions import train, evaluate, inference

In [ ]:
import sys
sys.path.append(os.path.expanduser('../ocpmodels/models'))
sys.path.append(os.path.expanduser('../../ocp_airi'))

from spinconv_with_embeds_single import spinconv

In [ ]:
#вызывается каждый раз, когда датасет отдаёт элемент (систему)
#делаем из данных матрицу векторов-атомов, список рёбер (edge_index) и матрицу векторов-рёбер; надо писать свою функцию для каждой сети
def preprocessing(system):
    keys = ['pos', 'atomic_numbers', 'cell', 'natoms']
    features_dict = {}
    for key in keys:
        features_dict[key] = system[key]
    return Data(**features_dict)

In [ ]:
#config
batch_size = 70
num_workers = 0

features_cols = ['feature_1']

target_col = 'y_relaxed'
lr = 0.001
epochs = 30

In [ ]:
# #чтобы тензор по умолчанию заводился на куде
# if torch.cuda.is_available():
#     torch.set_default_tensor_type('torch.cuda.FloatTensor')
#     print('cuda')

In [ ]:
#set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
print(device)

In [ ]:
#инициализируем тренировочный датасети и тренировочный итератор
train_dataset_file_path= os.path.expanduser("../../ocp_datasets/data/is2re/all/train/data.lmdb")
train_dataset_file = lmdb_dataset(train_dataset_file_path)
training_set = Dataset(train_dataset_file, features_cols, target_col, preprocessing=preprocessing)
training_generator = DataListLoader(training_set, batch_size=batch_size)

In [ ]:
#инициализируем валидационный датасет и валидационный итератор
val_dataset_file_path = os.path.expanduser("../../ocp_datasets/data/is2re/all/val_ood_both/data.lmdb")
val_dataset_file = lmdb_dataset(val_dataset_file_path)
valid_set = Dataset(val_dataset_file, features_cols, target_col, preprocessing=preprocessing)
valid_generator = DataListLoader(valid_set, batch_size=batch_size, num_workers=num_workers)

In [ ]:
try:
    lmdb_dataset(train_dataset_file_path).describe()
except:
    pass

In [ ]:

# #model
# model = spinconv(None, None, 1, otf_graph=True, regress_forces=False, 0)
# model = DataParallel(model)
# model = model.to(device)

# #optimizer and loss
# optimizer = optim.AdamW(model.parameters(), lr=lr)
# criterion = nn.L1Loss()

# #переносим на куду если она есть
# criterion = criterion.to(device)

In [ ]:
timestamp = str(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
timestamp = '2021-10-05-14-40-54'

print(timestamp)

In [ ]:
#tensorboard writer, при первом запуске надо руками сделать папку для логов

# server
#log_folder_path = "../../ocp_results/logs/tensorboard/out_base_model"

# colab
# log_folder_path = "/content/drive/MyDrive/ocp_results/logs/tensorboard/out_base_model"

# user_specific 
log_file_path = "../../logs/tensorboard_airi"

writer = SummaryWriter(log_file_path + '/' + timestamp)

In [ ]:
%%time
logfile_str = {
    "train_dataset_file_path": train_dataset_file_path,
    "val_dataset_file_path": val_dataset_file_path,
    "features_cols": features_cols,
    "target_col": target_col,
    "batch_size": batch_size,
    "num_workers": num_workers,
    "epochs": epochs,
    "lr": lr,
    "type":'id'
}

#граф модели
try:
    #trace_system = dict(list(next(iter(training_generator))[0]))
    writer.add_graph(model, trace_system)
except:
    print('no graph')
writer.add_text(timestamp, str(logfile_str))

In [ ]:
%ls ../logs/tensorboard_airi

## Training

In [ ]:
# %%time
# loss = []
# loss_eval = []

# print(timestamp)
# print(f'Start training model {str(model)}')
# for i in range(epochs):
#     loss.append(train(model, training_generator, optimizer, criterion, epoch=i, writer=writer, device=device))
#     loss_eval.append(evaluate(model, valid_generator, criterion, epoch=i, writer=writer, device=device))

In [ ]:
embed_discr = {"group_onehot" : list(range(18)),
               "period_onehot": list(range(19, 27)),
               "block_onehot" : list(range(28, 32)),
               "electronegativity" : 33,
               "radius" : 34,
               "valence" : 35,
               "ionization" : 36,
               "affinity" : 37,
               "volume": 38
              }


#model


model_path = '../../ocp_airi/airi_utils/checkpoint/2021-10-05-14-40-54_epoch_0_.pickle'
model_state_dict = '../../ocp_airi/airi_utils/checkpoint/2021-10-05-14-40-54_epoch_0_state_dict.pickle'

# model = torch.load(model_path)

model = spinconv(None, None, 1, otf_graph=True, regress_forces=False, custom_embedding_value=torch.tensor(list(range(27))))
model = DataParallel(model)
model = model.to(device)
model.load_state_dict(torch.load(model_state_dict))

In [ ]:
#optimizer and loss
optimizer = optim.AdamW(model.parameters(), lr=lr)
criterion = nn.L1Loss()

#переносим на куду если она есть
criterion = criterion.to(device)
# timestamp = str(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
print(timestamp)

log_file_path = "../logs/tensorboard_airi"

writer = SummaryWriter(log_file_path + '/' + timestamp)
# %%time
logfile_str = {
    "train_dataset_file_path": train_dataset_file_path,
    "val_dataset_file_path": val_dataset_file_path,
    "features_cols": features_cols,
    "target_col": target_col,
    "batch_size": batch_size,
    "num_workers": num_workers,
    "epochs": epochs,
    "lr": lr,
    "type":'id',
    "custom_embedding_type": "group+period"
}

#граф модели
try:
    #trace_system = dict(list(next(iter(training_generator))[0]))
    writer.add_graph(model, trace_system)
except:
    print('no graph')
writer.add_text(timestamp, str(logfile_str))
# %%time
loss = []
loss_eval = []

print(timestamp)
print(f'Start training model {str(model)}')
for i in range(1, epochs):
    loss.append(train(model, training_generator, optimizer, criterion, epoch=i, writer=writer, device=device))
    loss_eval.append(evaluate(model, valid_generator, criterion, epoch=i, writer=writer, device=device))
    path = '_'.join((timestamp, 'epoch', str(i), '.pickle'))
    torch.save(model, path)

In [ ]:
# torch.save(model.state_dict(), '../../ocp_airi/airi_utils/checkpoint/2021-10-05-14-40-54_epoch_0_state_dict.pickle')